In [312]:
import pandas as pd
import numpy as np
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [313]:
def get_lol_heros_urls():
    hero_summary = []
    # Get html by using requests
    url = "https://champion.gg/"
    r = requests.get(url, timeout=30).text
    # Use BeautifulSoup to parse the content
    soup = BeautifulSoup(r, "html.parser")
    # Find the heros information
    table = soup.find(name="div", attrs={"class": "col-md-9 clearfix"})
    for hero in table.find_all(name="div", attrs={"class": "champ-height"}):
        # Get hero name
        s = []
        name = hero.find(name="span", attrs={"class": "champion-name"}).string
        name = name.split("&")[0]
        s.append(name.replace(" ", "").replace(".", ""))
        # Get lane of the hero
        for lane in hero.find_all(name="a")[1:]:
            l = lane.string
            l = l.replace(" ", "").replace("\n", "")
            s.append(l)
        hero_summary.append(s)
    urls = []
    basic_url = "https://champion.gg/champion/"
    for i in range(len(hero_summary)):
        for lane in hero_summary[i][1:]:
            url = basic_url + str(hero_summary[i][0]) + "/" + str(lane)
            urls.append(url)
    return urls

In [314]:
def get_statistics(url):
    hero_name = url.split("/")[-2]
    lane = url.split("/")[-1]
    hero_stat = []
    hero_stat.append(hero_name)
    hero_stat.append(lane)

    r = requests.get(url, timeout=30).text
    soup = BeautifulSoup(r, "html.parser")
    tbody = soup.find(name="tbody")
    if tbody is not None:
        for tr in tbody.find_all(name="tr", limit=11):
            # Find average score in every rows
            tds = tr.find_all(name="td")
            if tds[1] is not None:
                average = tds[1].string
                average = str(average).replace(" ", "").replace("\n", "")
            hero_stat.append(average)

            # Find role placement ratio in every rows
            tds = tr.find_all(name="td")
            if tds[2].find(name="strong") is not None:
                role_placement_1st = tds[2].find(name="strong").string
                role_placement_1st = str(role_placement_1st).replace(" ", "").replace("\n", "")
            if tds[2].find(name="small") is not None: 
                role_placement_2nd = tds[2].find(name="small").string
                role_placement_2nd = str(role_placement_2cd).replace(" ", "").replace("/", "")
            role_placement = role_placement_1st + "/" + role_placement_2cd
            hero_stat.append(role_placement)

        tr = tbody.find_all(name="tr")
        overall_placement_1st = tr[-1].find(name="strong").string.replace(" ", "").replace("\n", "")
        overall_placement_2nd = tr[-1].find(name="small").string.replace(" ", "").replace("/", "")
        overall_placement = overall_placement_1st + "/" + overall_placement_2nd
        hero_stat.append(overall_placement)
    
    return hero_stat

In [315]:
def get_hero_dataframe():
    stat = []
    urls = get_lol_heros_urls()
    for url in tqdm(urls):
        hero_stat = get_statistics(url)
        stat.append(hero_stat)
        df = pd.DataFrame(stat)
    df.columns = ["hero", "lane", 
       "win_rate_average", "win_rate_rp", 
       "play_rate_average", "play_rate_rp", 
       "ban_rate_average", "ban_rate_rp", 
       "player_base_average_games_played_average", "player_base_average_games_played_rp", 
       "gold_earned_average", "gold_earned_rp", 
       "kills_average", "kills_rp", 
       "deaths_average", "death_rp", 
       "assists_average", "assists_rp", 
       "damage_dealt_average", "damage_dealt_rp", 
       "damage_taken_average", "amage_taken_rp", 
       "minions_killed_average", "minions_killed_rp", 
       "overall_placement_rp"]
    return df

In [316]:
df = get_hero_dataframe()
df

,hero,lane,win_rate_average,win_rate_rp,play_rate_average,play_rate_rp,ban_rate_average,ban_rate_rp,player_base_average_games_played_average,player_base_average_games_played_rp,...,death_rp,assists_average,assists_rp,damage_dealt_average,damage_dealt_rp,damage_taken_average,amage_taken_rp,minions_killed_average,minions_killed_rp,overall_placement_rp
0,Aatrox,Top,47.26%,37/44,3.00%,11/44,0.59%,16/44,5.30,30/44,...,21/44,5.80,22/44,18796,24/44,31737,8/44,161.86,30/44,23/44
1,Aatrox,Support,50.68%,22/44,0.55%,41/44,0.59%,15/44,23.03,5/44,...,1/44,2.06,47/44,8707,29/44,13291,31/44,100.40,2/44,36/48
2,Ahri,Middle,51.90%,7/44,3.13%,9/44,0.23%,20/44,7.55,10/44,...,7/44,7.68,14/44,20122,24/44,18569,28/44,159.56,27/44,14/42
3,Akali,Middle,44.25%,42/44,1.65%,23/44,0.48%,13/44,7.93,9/44,...,32/44,5.34,42/44,18860,34/44,24557,8/44,149.10,36/44,42/42
4,Akali,Top,42.43%,44/44,0.88%,34/44,0.48%,18/44,8.23,5/44,...,33/44,4.76,37/44,19085,17/44,25005,31/44,160.67,31/44,43/44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Zilean,Support,50.84%,18/44,2.17%,22/44,0.12%,35/44,7.05,24/44,...,13/44,11.35,17/44,10550,18/44,13714,28/44,38.16,24/44,19/48
202,Zilean,Middle,55.09%,1/44,0.43%,39/44,0.12%,25/44,8.40,7/44,...,1/44,10.73,1/44,16304,42/44,18091,30/44,162.12,24/44,33/42
203,Zoe,Middle,47.16%,37/44,2.29%,15/44,0.47%,14/44,NaN,25/44,...,16/44,7.23,18/44,22084,10/44,17939,31/44,153.65,33/44,27/42
204,Zoe,Support,48.94%,38/44,0.45%,45/44,0.47%,18/44,NaN,36/44,...,5/44,3.75,42/44,9480,23/44,9192,45/44,87.34,7/44,42/48


In [317]:
df.isnull().sum()

hero                                        0
lane                                        0
win_rate_average                            0
win_rate_rp                                 0
play_rate_average                           0
play_rate_rp                                0
ban_rate_average                            0
ban_rate_rp                                 0
player_base_average_games_played_average    0
player_base_average_games_played_rp         0
gold_earned_average                         0
gold_earned_rp                              0
kills_average                               0
kills_rp                                    0
deaths_average                              0
death_rp                                    0
assists_average                             0
assists_rp                                  0
damage_dealt_average                        0
damage_dealt_rp                             0
damage_taken_average                        0
amage_taken_rp                    

In [305]:
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()
df[df.isnull().any(axis=1)][null_columns].head()

,win_rate_average,win_rate_rp,play_rate_average,play_rate_rp,ban_rate_average,ban_rate_rp,player_base_average_games_played_average,player_base_average_games_played_rp,gold_earned_average,gold_earned_rp,...,death_rp,assists_average,assists_rp,damage_dealt_average,damage_dealt_rp,damage_taken_average,amage_taken_rp,minions_killed_average,minions_killed_rp,overall_placement_rp
29,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
30,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
104,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [320]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 25 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   hero                                      206 non-null    object
 1   lane                                      206 non-null    object
 2   win_rate_average                          206 non-null    object
 3   win_rate_rp                               206 non-null    object
 4   play_rate_average                         206 non-null    object
 5   play_rate_rp                              206 non-null    object
 6   ban_rate_average                          206 non-null    object
 7   ban_rate_rp                               206 non-null    object
 8   player_base_average_games_played_average  206 non-null    object
 9   player_base_average_games_played_rp       206 non-null    object
 10  gold_earned_average                       206 non-